In [50]:
%matplotlib widget

%load_ext autoreload
%autoreload 1

from pathlib import Path
import re

import numpy as np
import pandas as pd
from pandas.plotting import register_matplotlib_converters
import matplotlib.pyplot as plt

%aimport vis


register_matplotlib_converters()

Two runs of Auto-PyTorch with 20 budgets and 2 iterations

In [11]:
df = pd.read_csv('log/2019-09-22T20:35:20/jobs.csv')
# df = pd.read_csv('log/2019-09-23T10:57:58/jobs.csv', sep='\t')
df['StartTime'] = pd.to_datetime(df['Start Time'])
df['EndTime'] = pd.to_datetime(df['End Time'])
df['Duration'] = df.EndTime - df.StartTime
df['Worker'] = pd.to_numeric(df.Worker)

ax = vis.job_timeline(df.Worker, df.StartTime, df.EndTime)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

One run of hyperopt with 300 max eval

In [20]:
total = pd.DataFrame()
for node in range(1, 5):
    df = pd.read_csv(f'log/2019-09-30T10:04:29/hyper{node}.csv', sep='\t',
                     names=['StartTime', 'EndTime', 'Node'])
    total = total.append(df)
total = total.reset_index(drop=True)
total['Node'] = pd.to_numeric(total.Node)
total['StartTime'] = pd.to_datetime(total.StartTime)
total['EndTime'] = pd.to_datetime(total.EndTime)
total = total.reset_index(drop=True)

fig, ax = plt.subplots(figsize=(12, 12))
ax = vis.job_timeline(total.Node, total.StartTime, total.EndTime, ax=ax)
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

No handles with labels found to put in legend.


---

One run of Auto-PyTorch with 200 budget and 10 iterations.

It seems that the algorithm can finish earlier than max iterations

In [23]:
total = pd.DataFrame()
for node in range(1, 6):
    df = pd.read_csv(f'log/2019-09-30T10:04:29/node{node}.csv', sep='\t',
                     names=['Iter', 'Unknown', 'JobId', 'StartTime', 'EndTime'])
    df['Node'] = node
    total = total.append(df)
total['StartTime'] = pd.to_datetime(total.StartTime)
total['EndTime'] = pd.to_datetime(total.EndTime)
total = total.reset_index(drop=True)

In [51]:
fig, ax = plt.subplots()
vis.job_timeline(total.Node, total.StartTime, total.EndTime, groupby=total.Iter, label='Iter{key}', ax=ax)
vis.figure_legend(fig, ncol=5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [61]:
total['Duration'] = total.EndTime - total.StartTime
total['Duration'] = total.Duration.astype('timedelta64[s]')

In [62]:
total.Duration

0         8.0
1        19.0
2         2.0
3         3.0
4         3.0
        ...  
369     594.0
370     133.0
371      53.0
372    3403.0
373      30.0
Name: Duration, Length: 374, dtype: float64

In [75]:
fig, axs = plt.subplots(nrows=3, ncols=3, sharex=True, constrained_layout=True)
grouped = total.groupby('Iter')
for ax, (it, grp) in zip(axs.flat, total.groupby('Iter')):
    ax.hist(grp.Duration)
    ax.set_title(f'Iter {it}')

/home/peifeng/.local/venvs/autopytorch/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …